#### Below is a plot of the baseline hazard function which includes the decreasing risk the longer they survive (the Lindy effect)

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Define the function
def baseline_hazard(t):
    return 0.001 * np.exp(-0.25 * t)

# Define the time array
t = np.linspace(0, 10, 100)

# Calculate the baseline hazard values
baseline_hazard_values = baseline_hazard(t)

# Create the plot
plt.plot(t, baseline_hazard_values)
plt.xlabel('Time')
plt.ylabel('Baseline Hazard')
plt.title('Baseline Hazard Function')
plt.show()
